In [1]:
import pandas

pandas.set_option('display.max_columns', None)
pandas.set_option('display.max_rows', 25)

In [12]:
import os

# Set up some variables for the training data paths
BASE_DATA_PATH = os.path.join('..', 'data')
TRAIN_DATA_PATH = os.path.join(BASE_DATA_PATH, 'train')

TRAINING_PROVIDERS_PATH = os.path.join(TRAIN_DATA_PATH, 'Train-1542865627584.csv')
TRAINING_INPATIENT_PATH = os.path.join(TRAIN_DATA_PATH, 
    'Train_Inpatientdata-1542865627584.csv')
TRAINING_OUTPATIENT_PATH = os.path.join(TRAIN_DATA_PATH, 
    'Train_Outpatientdata-1542865627584.csv')
TRAINING_BENEFICIARY_PATH = os.path.join(TRAIN_DATA_PATH, 
    'Train_Beneficiarydata-1542865627584.csv')

# Defines the default behavior for responding to values that do not exist in data set
DEFAULT_NA_VALUE = "None"


In [13]:
# Read all of the training data
train_providers = pandas.read_csv(TRAINING_PROVIDERS_PATH)
train_inpatient = pandas.read_csv(TRAINING_INPATIENT_PATH).fillna(DEFAULT_NA_VALUE)
train_outpatient = pandas.read_csv(TRAINING_OUTPATIENT_PATH).fillna(DEFAULT_NA_VALUE)
train_beneficiary = pandas.read_csv(TRAINING_BENEFICIARY_PATH).fillna(DEFAULT_NA_VALUE)